# Deploy Watson ML (PMML)

Deploys a PMML model to IBM Watson Machine Learning (WML)

In [1]:
from ibm_watson_machine_learning import APIClient
import logging
import os
import re
import sys

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# IBM Cloud API Key https://cloud.ibm.com/iam/apikeys
api_key = os.environ.get('api_key', 'kCGXtWuyZ1JIBOJeIyidz0eCOh4D7oRd0vK6aZ5vuho6')

# Machine Learning Model Deployment Space https://dataplatform.cloud.ibm.com/ml-runtime/spaces
space = os.environ.get('space', '7ff745a2-84e7-4a7d-83a6-388e701e9474')

# IBM Cloud Region (e.g. us-south)
location = os.environ.get('location', 'us-south')

# temporary directory for data
data_dir = os.environ.get('data_dir',
                          'data/')

In [3]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [4]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [5]:
client = APIClient(wml_credentials)

In [6]:
o = client.software_specifications.get_uid_by_name('pmml-3.0_4.3')
software_spec_uid = o
client.set.default_space(space)

'SUCCESS'

In [8]:
model_meta_props = {
    client.repository.ModelMetaNames.NAME: 'test_pmml2',
    client.repository.ModelMetaNames.TYPE: "pmml_4.2",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

published_model = client.repository.store_model(
    model=data_dir + 'model.xml',
    meta_props=model_meta_props,
)

model_uid = client.repository.get_model_id(published_model)